# 4.1 データセットと前処理

このノートブックでは、第4.1節の内容に沿って、事前学習用のデータセット前処理パイプラインを実装します。

- データ取得（Hugging Face datasets: globis-university/aozorabunko-clean）
- テキスト正規化（neologdn + 全角半角変換）
- 文書境界セパレータ付与＆連結
- 学習用チャンク化（2048文字）
- HuggingFace Dataset形式での保存

トークナイザの学習・保存は4.3節で実施します。

In [1]:
from __future__ import annotations

import re
from pathlib import Path

import neologdn
from datasets import Dataset, DatasetDict, load_dataset

## 設定

In [2]:
# 出力先ディレクトリ
OUTPUT_DIR = Path("data")

# 本文テキストのカラム名
TEXT_COL = "text"

# 文書境界セパレータ
SEP = "\n\n<|doc|>\n\n"

# チャンクサイズ（文字数）
BLOCK_SIZE = 2048

## データ取得（Hugging Face datasets）

`globis-university/aozorabunko-clean` を取得します。青空文庫のテキストをもとにしたデータセットで、グロービス経営大学院が CC-BY ライセンスで公開しています。

In [3]:
raw: DatasetDict = load_dataset("globis-university/aozorabunko-clean")
print(f"データセット構成: {raw}")
print(f"train split のサンプル数: {len(raw['train'])}")

README.md: 0.00B [00:00, ?B/s]

aozorabunko-dedupe-clean.jsonl.gz:   0%|          | 0.00/241M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16951 [00:00<?, ? examples/s]

データセット構成: DatasetDict({
    train: Dataset({
        features: ['text', 'footnote', 'meta'],
        num_rows: 16951
    })
})
train split のサンプル数: 16951


In [4]:
# サンプルテキストを確認
print(raw["train"][0][TEXT_COL][:200])

深いおどろきにうたれて、
名高いウェストミンスターに
真鍮や石の記念碑となって
すべての王侯貴族が集まっているのをみれば、
今はさげすみも、ほこりも、見栄もない。
善にかえった貴人の姿、
華美と俗世の権勢をすてた
けがれのない帝王の姿がみえるではないか。
いろどられた、おもちゃのような墓石に
今は静かに物云わぬ魂がどんなに満足していることか。
かつてはその足にふまえた全世界をもってしても
その欲望


## テキスト正規化

日本語テキストには表記揺れが多く存在します。例えば：
- ダッシュ `――` ↔ 長音符 `ー`
- 全角英字 `Ｔ・Ｂ` ↔ 半角英字 `T・B`

この種の差異は**語彙の分裂**を招き、学習効率を落とします。そこで、日本語向け正規化ライブラリ `neologdn` を用いて記号と全半角を機械学習寄りに揃えます。

> 注意：文学テキストでは過度な正規化が作風（ダッシュのリズム、三点リーダ等）を損なうことがある。ここでは言語モデルの事前学習という目的を優先し、可読性より統計的一貫性を重視する。

In [5]:
def normalize_ja(text: str) -> str:
    """日本語テキストの正規化
    
    Args:
        text: 入力テキスト
    
    Returns:
        正規化されたテキスト
    """
    # 1. neologdnによるデフォルト正規化
    normalized_text = neologdn.normalize(text)

    # 2. 全角英数 → 半角（A-Z/a-z/0-9 のみ最小実装）
    def z2h_alnum(match):
        ch = match.group(0)
        return chr(ord(ch) - 0xFEE0)
    normalized_text = re.sub(r"[Ａ-Ｚａ-ｚ０-９]", z2h_alnum, normalized_text)

    # 3. クオートの半角化
    normalized_text = normalized_text.replace("＂", '"').replace("＇", "'")

    # 4. 三点リーダの統一
    normalized_text = normalized_text.replace("･･･", "…").replace("・・・", "…")

    # 5. 連続空白の圧縮
    normalized_text = re.sub(r"\s+", " ", normalized_text).strip()

    return normalized_text

In [6]:
# 正規化の動作確認
test_text = "――「クリストレロの諷刺詩」一五九八年、Ｔ・Ｂ作"
print(f"正規化前: {test_text}")
print(f"正規化後: {normalize_ja(test_text)}")

正規化前: ――「クリストレロの諷刺詩」一五九八年、Ｔ・Ｂ作
正規化後: ー「クリストレロの諷刺詩」一五九八年、T・B作


In [7]:
def apply_normalize(batch: dict) -> dict:
    """バッチ処理用の正規化関数"""
    texts = batch[TEXT_COL]
    return {TEXT_COL: [normalize_ja(t) for t in texts]}

# データセット全体に適用（batched=True で速度改善）
norm_ds = raw.map(
    apply_normalize,
    batched=True,
    desc="正規化処理",
)

print(f"正規化後のデータセット: {norm_ds}")

正規化処理:   0%|          | 0/16951 [00:00<?, ? examples/s]

正規化後のデータセット: DatasetDict({
    train: Dataset({
        features: ['text', 'footnote', 'meta'],
        num_rows: 16951
    })
})


In [8]:
# 正規化後のサンプルを確認
print(norm_ds["train"][0][TEXT_COL][:200])

深いおどろきにうたれて、 名高いウェストミンスターに 真鍮や石の記念碑となって すべての王侯貴族が集まっているのをみれば、 今はさげすみも、ほこりも、見栄もない。 善にかえった貴人の姿、 華美と俗世の権勢をすてた けがれのない帝王の姿がみえるではないか。 いろどられた、おもちゃのような墓石に 今は静かに物云わぬ魂がどんなに満足していることか。 かつてはその足にふまえた全世界をもってしても その欲望


## 文書境界セパレータ付与＆連結

事前学習用に、文書間の境界を示すセパレータ `<|doc|>` を挿入しながら全文書を連結します。

In [9]:
def make_long_string(dsdict: DatasetDict, split: str, col: str = TEXT_COL) -> str:
    """データセットの全文書をセパレータで連結
    
    Args:
        dsdict: データセット辞書
        split: 使用するsplit名（'train'など）
        col: テキストカラム名
    
    Returns:
        連結されたテキスト
    """
    texts = dsdict[split][col]
    return SEP.join(texts)

long_text = make_long_string(norm_ds, split="train")
print(f"連結後の総文字数: {len(long_text):,}")

連結後の総文字数: 227,624,821


In [10]:
# 連結結果のサンプルを確認（セパレータが含まれていることを確認）
sep_pos = long_text.find(SEP)
if sep_pos != -1:
    print(f"最初のセパレータ位置: {sep_pos}")
    print(f"セパレータ周辺のテキスト:\n{long_text[sep_pos-50:sep_pos+50]}")

最初のセパレータ位置: 10605
セパレータ周辺のテキスト:
。その生涯ははかない物語のようであり、その記念碑さえも廃墟となるのである。 原註トマス・ブラウン卿。

<|doc|>

いざ、これより樂しまむ、 仕置を受くる憂なく、 遊びたのしむ時ぞ來ぬ、 時ぞ來


## 学習用チャンク化

連結したテキストを固定長（2048文字）のチャンクに分割します。これはトークナイズ前の「プレトークナイズ」処理です。

In [11]:
def chunk_text(s: str, size: int):
    """テキストを固定長のチャンクに分割
    
    Args:
        s: 入力テキスト
        size: チャンクサイズ（文字数）
    
    Yields:
        チャンク化されたテキスト
    """
    for i in range(0, len(s), size):
        yield s[i : i + size]

# 文字列をBLOCK_SIZE文字ごとに分割
chunks = list(chunk_text(long_text, BLOCK_SIZE))
print(f"チャンク数: {len(chunks):,}")
print(f"最初のチャンクの長さ: {len(chunks[0])}")
print(f"最後のチャンクの長さ: {len(chunks[-1])}")

チャンク数: 111,145
最初のチャンクの長さ: 2048
最後のチャンクの長さ: 1909


In [12]:
# 最初のチャンクを確認
print(chunks[0][:500])

深いおどろきにうたれて、 名高いウェストミンスターに 真鍮や石の記念碑となって すべての王侯貴族が集まっているのをみれば、 今はさげすみも、ほこりも、見栄もない。 善にかえった貴人の姿、 華美と俗世の権勢をすてた けがれのない帝王の姿がみえるではないか。 いろどられた、おもちゃのような墓石に 今は静かに物云わぬ魂がどんなに満足していることか。 かつてはその足にふまえた全世界をもってしても その欲望を満たすこともおさえることも出来なかったのに。 生とは冷たい幸福の結ぶ氷であり、 死とはあらゆる人間の虚栄をとかす霜解けである。 ー「クリストレロの諷刺詩」一五九八年、T・B作 秋も更けて、暁闇がすぐに黄昏となり、暮れてゆく年に憂愁をなげかけるころの、おだやかな、むしろ物さびしいある日、わたしはウェストミンスター寺院を逍遥して数時間すごしたことがある。悲しげな古い大伽藍の荘厳さには、この季節の感覚になにかぴったりするものがあった。その入口を通ったとき、わたしは、昔の人の住む国に逆もどりし、過ぎ去った時代の闇のなかに身を没してゆくような気がした。 わたしはウェストミンスター・スクールの中庭から入


## HuggingFace Dataset形式で保存

チャンク化したデータをHuggingFace Dataset形式で保存します。これにより、4.3節の事前学習でそのまま利用できます。

In [13]:
# Dataset形式化
out_ds = Dataset.from_dict({"text": chunks})
print(f"出力データセット: {out_ds}")

出力データセット: Dataset({
    features: ['text'],
    num_rows: 111145
})


In [14]:
# 保存
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
out_ds.save_to_disk(OUTPUT_DIR / "aozora_preprocessed")
print(f"保存先: {OUTPUT_DIR / 'aozora_preprocessed'}")

Saving the dataset (0/2 shards):   0%|          | 0/111145 [00:00<?, ? examples/s]

保存先: data/aozora_preprocessed


## 保存データの確認

In [15]:
from datasets import load_from_disk

# 保存したデータを読み込んで確認
loaded_ds = load_from_disk(OUTPUT_DIR / "aozora_preprocessed")
print(f"読み込んだデータセット: {loaded_ds}")
print(f"\nサンプルテキスト:\n{loaded_ds[0]['text'][:300]}")

読み込んだデータセット: Dataset({
    features: ['text'],
    num_rows: 111145
})

サンプルテキスト:
深いおどろきにうたれて、 名高いウェストミンスターに 真鍮や石の記念碑となって すべての王侯貴族が集まっているのをみれば、 今はさげすみも、ほこりも、見栄もない。 善にかえった貴人の姿、 華美と俗世の権勢をすてた けがれのない帝王の姿がみえるではないか。 いろどられた、おもちゃのような墓石に 今は静かに物云わぬ魂がどんなに満足していることか。 かつてはその足にふまえた全世界をもってしても その欲望を満たすこともおさえることも出来なかったのに。 生とは冷たい幸福の結ぶ氷であり、 死とはあらゆる人間の虚栄をとかす霜解けである。 ー「クリストレロの諷刺詩」一五九八年、T・B作 秋も更けて、暁闇がすぐ


## まとめ

このノートブックでは、以下の前処理パイプラインを実装しました：

1. **データ取得**: Hugging Face datasets から青空文庫データを取得
2. **テキスト正規化**: neologdn + 全角半角変換で表記揺れを統一
3. **文書連結**: `<|doc|>` セパレータで全文書を連結
4. **チャンク化**: 2048文字単位で分割
5. **保存**: HuggingFace Dataset形式で保存

次の4.3節では、このデータセットを用いてトークナイザの学習と事前学習を行います。